In [1]:
import pandas as pd   # manipulacao de dados do CSV
import numpy as np    # algebra linear e calculos em geral

Nosso próximo passo é carregar os arquivos usando pandas.

In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
submission_df = pd.read_csv('../input/sample_submission.csv')

Vamos dar uma olhada nesses arquivos? O primeiro ponto importante é sabermos que tipo de informação nós temos.

In [3]:
train_df['name'] = train_df['name'].fillna('')
test_df['name'] = test_df['name'].fillna('')

In [4]:
train_df.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,18,1
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,1,0


Legal. O arquivo test_df tem o mesmo format, exceto que não tem a coluna de author. É o que estamos querendo prever, certo?

In [5]:
test_df.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,1449619185,1449619185,1446002581,1446159585
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,1453435620,1453435620,1450297323,1450411620
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,1451780700,1451780700,1448581356,1448672128
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,False,US,USD,1445021518,1445021530,1440966830,1442429518
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,False,US,USD,1462068840,1462068844,1455765276,1458334890


O arquivo de submission é um exemplo de como devemos mandas as previsões. Vamos usar ele como template mais pra frente. O importante aqui é notar que na submissão, a previsão é baseada em probabilidades.

In [6]:
submission_df.head()

,project_id,final_status
0,kkst917493670,0
1,kkst1664901914,0
2,kkst925125077,1
3,kkst1427645275,0
4,kkst1714249266,0


Mais um passo importante na hora de entendermos os dados é saber se o nosso dataset está equilibrado. Isso pode alterar o nosso tipo de abordagem de classificação.

In [7]:
train_df['final_status'].value_counts(normalize=True)

0    0.680373
1    0.319627
Name: final_status, dtype: float64

## Preparação dos Dados

O sklear, que é a biblioteca de ML que vamos usar, trabalha apenas com dados numéricos. Logo, vamos ter que dar uma massageada nos dados, porque tudo o que temos no CSV é texto. Lembra o que vimos lá em cima?

### Bag of Words
O primeiro passo que vamos fazer é transformar o texto de cada linha em uma representação de Bag of Words. Isso vai incluir o processo de tokenização.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()

Vamos pedir para que o CountVectorizer "aprenda" o vocabulário do nosso texto. Isso vai fazer com que ele seja capaz de trabalhar com Bag of Words.

*Importante*: veja que estamos ensinando o vocabulário usando os dois datasets: train e test! Isso é importante porque pode ser que existam palavras no dataset de test que não exista no dataset de treino.

In [9]:
bow.fit(test_df.append(train_df)['name'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Ótimo! Agora ele conhece o vocabulário. Quer ver?

Vamos começar espiando o testo da primeira amostra do dataset de treino.

In [10]:
print(bow.vocabulary_['process'])
print(bow.vocabulary_['this'])

60397
76885


In [11]:
print("Tamanho do Vocabulario aprendido:", len(bow.vocabulary_))

Tamanho do Vocabulario aprendido: 87042


Nesse processo as palavras foram todas convertidas para minusculas

In [12]:
'this' in bow.vocabulary_

True

Ótimo. Agora que já temos um vocabulário, hora de transformar o nosso texto em números, usando a representação de Bag of Words. O resultado vai ser uma matriz em que cada linha é uma amostra (alinhada com o dataset de treinamento) e cada coluna representa o número de vezes que aquela palavra apareceu.

In [13]:
train_X_bow = bow.transform(train_df['name'])
train_X_bow

<108129x87042 sparse matrix of type '<class 'numpy.int64'>'
	with 568885 stored elements in Compressed Sparse Row format>

É um pouco difícil de trabalhar com matrizes esparsas. Sabemos que essa matriz tem 19579 linhas (são as amostras) e 28300 colunas (são as palavras). Vamos espiar uma linha pra entender melhor o que está acontecendo? Vamos espiar a primeira linha.

In [14]:
x_bow_0 = train_X_bow[0].toarray().reshape(-1)
x_bow_0

array([0, 0, 0, ..., 0, 0, 0])

Ainda difícil de ver. Vamos reordenar, pra ficar mais fácil. Vamos ordenar em ordem decrescente, para saber quais as palavras mais frequentes nesse texto. Pelo modelo de bag of words, essas seriam as mais importantes.

In [15]:
idx = np.argsort(-x_bow_0)[:20]
print(idx)
print(x_bow_0[idx])
print(np.array(bow.get_feature_names())[idx])

[28648 21751 22291 58033 58032 58031 58030 58029 58028 58027 58026 58024
 58023 58022 58021 58020 58019 58018 58017 58016]
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['for' 'dollars' 'drawing' 'phytl' 'phystix' 'physis' 'physique'
 'physiomobile' 'physiological' 'physio' 'physigraphics' 'physick'
 'physicians' 'physician' 'physicallyfriendlycooking' 'physically'
 'physical' 'physic' 'phyre' 'phyllis']


Fica aí a reflexão: você acha que essa ordenação de fato reflete a importância dessas palavras? Tem jeito de melhorar isso?

Dica: stop words e TF-IDF

### TF-IDF
TF-IDF para os íntimos, é a sigla de Term Frequency - Inverse Document Frequency. Trata-se de uma transformação sobre o modelo de Bag of Words para tentar resolver alguns dos problemas que vimos ali em cima.

Essa transformação faz uma mágica: ele diminui a importância de palavras que aparecem em muitos documentos (como the, of, etc) e aumenta a importância de palavras que são mais raras naquelo documento: ou seja - que provavelmente são mais alinhadas com o estilo do autor ou do assunto. Quer saber como calcular o TF-IDF? É fácil, mas a gente não vai tratar disso aqui. [https://en.wikipedia.org/wiki/Tf%E2%80%93idf]

Primeiro nós precisamos fazer o TF-IDF Transformer "entender" quais as palavras comuns e quais não são comuns.

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tfidf.fit(train_X_bow)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

Agora que ele já sabe quais as palavras comuns, hora de transformar o nosso bag of words

In [17]:
train_X_tfidf = tfidf.transform(train_X_bow)
train_X_tfidf

<108129x87042 sparse matrix of type '<class 'numpy.float64'>'
	with 568885 stored elements in Compressed Sparse Row format>

Note que o tamanho da matriz é exatamente o mesmo. Vamos espiar o conteúdo?

In [18]:
x_tfidf_0 = train_X_tfidf[0].toarray().reshape(-1)
x_tfidf_0

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

Ainda difícil de ver porque é bem esparso. Vamos ordenar?

In [19]:
idx = np.argsort(-x_tfidf_0)[:20]
print(idx)
print(x_tfidf_0[idx])
print(np.array(bow.get_feature_names())[idx])

[21751 22291 28648 58033 58032 58031 58030 58029 58028 58027 58026 58024
 58023 58022 58021 58020 58019 58018 58017 58016]
[ 0.74545808  0.60282096  0.28442775  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
['dollars' 'drawing' 'for' 'phytl' 'phystix' 'physis' 'physique'
 'physiomobile' 'physiological' 'physio' 'physigraphics' 'physick'
 'physicians' 'physician' 'physicallyfriendlycooking' 'physically'
 'physical' 'physic' 'phyre' 'phyllis']


E aí? Agora faz mais sentido esse critério de importância (ou característica) de cada autor?

Fica aqui mais uma reflexão: é sempre desejável termos essa representação

### LabelEncoding do target
O nome do autor também é texto. Vamos ter que dar um jeito de converter os nomes dos autores para valores numérico, porque é o jeito que o sklearn trabalhar. Os 3 textos que temos que transformar são EAP, HPL e MWS. 

O método que vamos usar é chamado de Label Encoding. Ou seja: a cada texto, vamos atribuir um inteiro. Como por exemplo: EAP = 0, HPL = 1 e MWS = 2.

Antes de atribuirmos, precisamos que o Encoder "aprenda" quais as categorias existentes.

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_df['final_status'])
le.classes_

array([0, 1])

Ótimo! Agora o LabelEncoder já sabe quais são as categorias que ele tem que mapear. Agora precisamos efetivamente converter a coluna author. Bora lá.

In [21]:
train_y = le.transform(train_df['final_status'])
train_y

array([1, 0, 0, ..., 1, 0, 0])

## Treinando um Modelo Preditivo
Agora que já temos os dados preparados e transformados em dados numéricos, podemos treinar o nosso modelo de machine learning.

Para essa competição, vamos usar um classificador que em geral tem uma boa performance trabalhando com texto. Ele é baseado em estatística bayesiana e é normalmente chamado de Naive Bayes (Naive porque ele acredita que os dados não tenham relação entre si...). Não vamos entrar em detalhes aqui do que é um modelo bayesiano. O que importrante pra gente: ele é bom em calcular probabilidades. Se ele sabe que 40% dos textos são do Edgard Alan Poe e que a palavra "ascertaining" tem 0,03% de chance de aparecer num texto do Poe e que "uniform" tem 0,01% de chance de aparecer num texto do Poe, um texto que tem as palavras "uniform" e "ascertaining" tem qual a probabilidade de ser um texto do Poe? E da Mary Shelley? E do Lovecraft?

Esse é o tipo de cálculo que esse classificador faz. Quer ver mais detalhes? https://en.wikipedia.org/wiki/Naive_Bayes_classifier

Curiosidade: os filtros de Spam funcionam exatamente com esse tipo de classificador.

In [22]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=1.0)

Para treinar o modelo, temos que passar pra ele as "features" ou características - nesse caso, TFIDF - e quais são os targets, pra que ele possa calcular as probabilidades.

In [23]:
nb.fit(train_X_tfidf, train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Modelo treinado. Com o modelo treinado, podemos começar usá-lo para fazer predições. Só de farra, vamos usar o próprio dataset de treinamento para fazer uma previsão e ver como ele se comporta.

In [24]:
y_pred = nb.predict(train_X_tfidf)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

Ou seja: ele previu que o primeiro texto é do Poe (0), o segundo também, ... e o último é do Lovecraft (1)

## Avaliando o Modelo
Mas e aí. Esse modelo é bom? Qual é a taxa de acerto?

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(train_y, y_pred)

# 0.73535314300511423

0.73535314300511423

In [26]:
submission_df.head()

,project_id,final_status
0,kkst917493670,0
1,kkst1664901914,0
2,kkst925125077,1
3,kkst1427645275,0
4,kkst1714249266,0


Nós conseguimos gerar essas probabilidades usando o método predict_proba.

In [27]:
y_pred_proba = nb.predict_proba(train_X_tfidf)
y_pred_proba

array([[ 0.67519512,  0.32480488],
       [ 0.7743881 ,  0.2256119 ],
       [ 0.75574768,  0.24425232],
       ..., 
       [ 0.75589252,  0.24410748],
       [ 0.77445629,  0.22554371],
       [ 0.84825048,  0.15174952]])

In [28]:
train_df['name_prod'] = y_pred_proba[:,0]
train_df[['project_id','name_prod']].to_csv('submission_name_train.csv', index=False)

## Preparando a Submissão

Hora de preparar a nossa submissão. Vamos pegar o nosso modelo e prever os resultados a partir do arquivo de teste. É assim que o Kaggle avalia o seu modelo: entendendo como ele se comporta em um conjunto de dados que não foi visto durante o treinamento. Antes de fazer a previsão, precisamos aplicar exatamente as mesmas transformações que fizemos no dataset de treinamento.

Começando com a conversão pra bag of words...

In [29]:
test_X_bow = bow.transform(test_df['name'])
test_X_bow

<63465x87042 sparse matrix of type '<class 'numpy.int64'>'
	with 327582 stored elements in Compressed Sparse Row format>

Aplicando o TF-IDF...

In [30]:
test_X_tfidf = tfidf.transform(test_X_bow)
test_X_tfidf

<63465x87042 sparse matrix of type '<class 'numpy.float64'>'
	with 327582 stored elements in Compressed Sparse Row format>

E, finalmente, fazendo a previsão. Vamos usar o predict_proba, porque é o que o Kaggle espera.

In [31]:
y_pred_proba = nb.predict_proba(test_X_tfidf)
y_pred_proba

array([[ 0.58834307,  0.41165693],
       [ 0.60159079,  0.39840921],
       [ 0.64807653,  0.35192347],
       ..., 
       [ 0.8964605 ,  0.1035395 ],
       [ 0.83210959,  0.16789041],
       [ 0.77436271,  0.22563729]])

In [32]:
submission_df.head()

,project_id,final_status
0,kkst917493670,0
1,kkst1664901914,0
2,kkst925125077,1
3,kkst1427645275,0
4,kkst1714249266,0


Agora precisamos atribuir os valores que foram previstos pelo modelo.

In [33]:
submission_df['name_prod'] = y_pred_proba[:,0]

O resultado final vai ser parecido com isso.

In [34]:
submission_df.head()

,project_id,final_status,name_prod
0,kkst917493670,0,0.588343
1,kkst1664901914,0,0.601591
2,kkst925125077,1,0.648077
3,kkst1427645275,0,0.852360
4,kkst1714249266,0,0.668133


In [36]:
submission_df[['project_id','name_prod']].to_csv('submission_name_test.csv', index=False)